In [29]:
print(2)

2


In [30]:
from pyspark.sql import SparkSession
spark=(SparkSession
       .builder
       .appName("SparkApp")
       .getOrCreate()
       )

In [31]:
spark

## Raw user

In [32]:
df1=spark.read.csv("/home/prashant/Downloads/financial_trans/users_data.csv",header=True)
df1.createOrReplaceTempView('users_data')

df1.limit(1).show()

+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
| id|current_age|retirement_age|birth_year|birth_month|gender|      address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|825|         53|            66|      1966|         11|Female|462 Rose Lane|   34.15|  -117.76|           $29278|       $59696|   $127613|         787|               5|
+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+



In [33]:
df1.write.mode("overwrite").parquet("./output/raw_users")

In [34]:
with open("/home/prashant/Downloads/query/raw_users.sql") as f:
    users=f.read()

In [35]:
print(users)

select 
    id,
    gender,
    to_date(concat(birth_year||'/'||birth_month||'/1'),'yyyy/M/d') as birth_date,
    address,
    cast(regexp_replace(yearly_income,'[$,]','') as decimal(18,3)) as yearly_income,
    cast(regexp_replace(total_debt,'[$,]','') as decimal(18,3)) total_debt,
    cast(credit_score as int),
    case
        when credit_score between 800 and 850 then 5
        when credit_score between 740 and 799 then 4
        when credit_score between 670 and 739 then 3
        when credit_score between 580 and 669 then 2
        when credit_score between 300 and 579 then 1
        else null
    end as credit_rating,
    cast(num_credit_cards as int),
    cast(retirement_age as int),
    cast(retirement_age - (2025-birth_year) as int) as years_for_retirement
from users_data



In [36]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install setuptools

Note: you may need to restart the kernel to use updated packages.


In [41]:

spark.sql(users).show()



+----+------+----------+--------------------+-------------+----------+------------+-------------+----------------+--------------+--------------------+
|  id|gender|birth_date|             address|yearly_income|total_debt|credit_score|credit_rating|num_credit_cards|retirement_age|years_for_retirement|
+----+------+----------+--------------------+-------------+----------+------------+-------------+----------------+--------------+--------------------+
| 825|Female|1966-11-01|       462 Rose Lane|    59696.000|127613.000|         787|            4|               5|            66|                   7|
|1746|Female|1966-12-01|3606 Federal Boul...|    77254.000|191349.000|         701|            3|               5|            68|                   9|
|1718|Female|1938-11-01|     766 Third Drive|    33483.000|   196.000|         698|            3|               5|            67|                 -20|
| 708|Female|1957-01-01|    3 Madison Street|   249925.000|202328.000|         722|           